HW7

Построить обобщенную линейную модель (GLM) для прогнозирования наступления страховых случаев на рассмотренных в ноутбуке данных. Придумать и использовать дополнительные факторы при построении модели (например, пересечения признаков и т.д.). Оценить результаты построенной модели при помощи различных метрик (можно использовать и другие метрики помимо представленных в ноутбуке), выявить проблемы. Попробовать улучшить результат и предложить способы решения обнаруженных проблем.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Загрузим набор данных

df = pd.read_csv('freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 20 columns):
Exposure             115155 non-null float64
LicAge               115155 non-null int64
RecordBeg            115155 non-null object
RecordEnd            59455 non-null object
Gender               115155 non-null object
MariStat             115155 non-null object
SocioCateg           115155 non-null object
VehUsage             115155 non-null object
DrivAge              115155 non-null int64
HasKmLimit           115155 non-null int64
BonusMalus           115155 non-null int64
ClaimAmount          115155 non-null float64
ClaimInd             115155 non-null int64
ClaimNbResp          115155 non-null float64
ClaimNbNonResp       115155 non-null float64
ClaimNbParking       115155 non-null float64
ClaimNbFireTheft     115155 non-null float64
ClaimNbWindscreen    115155 non-null float64
OutUseNb             115155 non-null float64
RiskArea             115155 non-null float64
dtypes

В предыдущем уроке мы заметили отрицательную величину убытка для некоторых наблюдений. Заметим, что для всех таких полисов переменная "ClaimInd" принимает только значение 0. Поэтому заменим все соответствующие значения "ClaimAmount" нулями.

In [3]:
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())
NegClaimAmount.shape

Unique values of ClaimInd: [0]


(967, 2)

In [4]:
NegClaimAmount.head()

,ClaimAmount,ClaimInd
82,-74.206042,0
175,-1222.585196,0
177,-316.288822,0
363,-666.758610,0
375,-1201.600604,0


In [5]:
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

Перекодируем переменные типа `object` с помощью числовых значений

In [6]:
def SeriesFactorizer(series):
    series, unique = pd.factorize(series)
    reference = {x: i for x, i in enumerate(unique)}
    print(reference)
    return series, reference

In [7]:
df.Gender, GenderRef = SeriesFactorizer(df.Gender)

{0: 'Male', 1: 'Female'}


In [8]:
df.MariStat, MariStatRef = SeriesFactorizer(df.MariStat)

{0: 'Other', 1: 'Alone'}


Для переменных, содержащих более 2 значений, различия между которыми не могут упорядочены, используем фиктивные переменные (one-hot encoding).

**NB**: В H2O не рекомендуется использовать one-hot encoding, поскольку данный фреймворк корректно работает с категориальными признаками, тогда как применение one-hot encoding приводит к неэффективности. Тем не менее, используем здесь фиктивные переменные, чтобы в дальнейшем сохранить возможность сравнения результатов построенных моделей.

In [9]:
list(df.VehUsage.unique())

['Professional', 'Private+trip to office', 'Private', 'Professional run']

In [10]:
VU_dummies = pd.get_dummies(df.VehUsage, prefix='VehUsg', drop_first=False)
VU_dummies.head()

,VehUsg_Private,VehUsg_Private+trip to office,VehUsg_Professional,VehUsg_Professional run
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0


Фактор "SocioCateg" содержит информацию о социальной категории в виде кодов классификации CSP. Агрегируем имеющиеся коды до 1 знака, а затем закодируем их с помощью one-hot encoding.

[Wiki](https://fr.wikipedia.org/wiki/Professions_et_cat%C3%A9gories_socioprofessionnelles_en_France#Cr%C3%A9ation_de_la_nomenclature_des_PCS)

[Более подробный классификатор](https://www.ast74.fr/upload/administratif/liste-des-codes-csp-copie.pdf)

In [11]:
df['SocioCateg'].unique()

array(['CSP50', 'CSP55', 'CSP60', 'CSP48', 'CSP6', 'CSP66', 'CSP1',
       'CSP46', 'CSP21', 'CSP47', 'CSP42', 'CSP37', 'CSP22', 'CSP3',
       'CSP49', 'CSP20', 'CSP2', 'CSP40', 'CSP7', 'CSP26', 'CSP65',
       'CSP41', 'CSP17', 'CSP57', 'CSP56', 'CSP38', 'CSP51', 'CSP59',
       'CSP30', 'CSP44', 'CSP61', 'CSP63', 'CSP45', 'CSP16', 'CSP43',
       'CSP39', 'CSP5', 'CSP32', 'CSP35', 'CSP73', 'CSP62', 'CSP52',
       'CSP27', 'CSP24', 'CSP19', 'CSP70'], dtype=object)

In [12]:
df['SocioCateg'] = df.SocioCateg.str.slice(0,4)

In [13]:
pd.DataFrame(df.SocioCateg.value_counts().sort_values()).rename({'SocioCateg': 'Frequency'}, axis=1)

,Frequency
CSP7,14
CSP3,1210
CSP1,2740
CSP2,3254
CSP4,7648
CSP6,24833
CSP5,75456


In [14]:
df = pd.get_dummies(df, columns=['VehUsage','SocioCateg'])

Теперь, когда большинство переменных типа `object` обработаны, исключим их из набора данных за ненадобностью.

In [15]:
df = df.select_dtypes(exclude=['object'])

Также создадим такую переменную, как квадрат возраста.

In [16]:
df['DrivAgeSq'] = df.DrivAge.apply(lambda x: x**2)
df.head()

,Exposure,LicAge,Gender,MariStat,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,...,VehUsage_Professional,VehUsage_Professional run,SocioCateg_CSP1,SocioCateg_CSP2,SocioCateg_CSP3,SocioCateg_CSP4,SocioCateg_CSP5,SocioCateg_CSP6,SocioCateg_CSP7,DrivAgeSq
0,0.083,332,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
1,0.916,333,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
2,0.550,173,0,0,32,0,68,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,1024
3,0.089,364,1,0,52,0,50,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,2704
4,0.233,426,0,0,57,0,50,0.0,0,0.0,...,0,0,0,0,0,0,0,1,0,3249


Для моделирования частоты убытков сгенерируем показатель как сумму индикатора того, что убыток произошел ("ClaimInd") и количества заявленных убытков по различным видам ущерба за 4 предшествующих года ("ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen").

В случаях, если соответствующая величина убытка равняется нулю, сгенерированную частоту также обнулим.

In [17]:
df['ClaimsCount'] = df.ClaimInd + df.ClaimNbResp + df.ClaimNbNonResp + df.ClaimNbParking + df.ClaimNbFireTheft + df.ClaimNbWindscreen
df.loc[df.ClaimAmount == 0, 'ClaimsCount'] = 0
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis=1, inplace=True)

In [18]:
pd.DataFrame(df.ClaimsCount.value_counts()).rename({'ClaimsCount': 'Policies'}, axis=1)

,Policies
0.0,104286
2.0,3529
1.0,3339
3.0,2310
4.0,1101
5.0,428
6.0,127
7.0,26
8.0,6
9.0,2


In [19]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "13.0.1" 2019-10-15; OpenJDK Runtime Environment (build 13.0.1+9); OpenJDK 64-Bit Server VM (build 13.0.1+9, mixed mode, sharing)
  Starting server from /usr/local/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/_1/v8k_p9n93ls8tlpzq5b5thsh0000gn/T/tmpx5wz_47n
  JVM stdout: /var/folders/_1/v8k_p9n93ls8tlpzq5b5thsh0000gn/T/tmpx5wz_47n/h2o_root1_started_from_python.out
  JVM stderr: /var/folders/_1/v8k_p9n93ls8tlpzq5b5thsh0000gn/T/tmpx5wz_47n/h2o_root1_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Europe/Moscow
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,13 days
H2O cluster name:,H2O_from_python_root1_5o6v73
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [20]:
h2o_df = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [21]:
from sklearn.model_selection import train_test_split

## * Домашнее задание: GLM для прогнозирования наступления страхового случая

In [22]:
# Разбиение датасета на train/val/test

x_train_ind, x_test_ind, y_train_ind, y_test_ind = train_test_split(df.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount'], axis=1), df.ClaimInd, test_size=0.3, random_state=1)
x_valid_ind, x_test_ind, y_valid_ind, y_test_ind = train_test_split(x_train_ind, y_train_ind, test_size=0.5, random_state=1)

In [23]:
# Преобразование в H2O-Frame

h2o_train_ind = h2o.H2OFrame(pd.concat([x_train_ind, y_train_ind], axis=1))
h2o_valid_ind = h2o.H2OFrame(pd.concat([x_valid_ind, y_valid_ind], axis=1))
h2o_test_ind = h2o.H2OFrame(pd.concat([x_test_ind, y_test_ind], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [24]:
# Преобразуем целевую переменную ClaimInd в категориальную при помощи метода asfactor во всех наборах данных

h2o_train_ind['ClaimInd'] = h2o_train_ind['ClaimInd'].asfactor()
h2o_valid_ind['ClaimInd'] = h2o_valid_ind['ClaimInd'].asfactor()
h2o_test_ind['ClaimInd'] = h2o_test_ind['ClaimInd'].asfactor()

In [25]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_binomial = H2OGeneralizedLinearEstimator(family = "binomial", nfolds=5)
glm_binomial.train(y="ClaimInd", x = h2o_train_ind.names[1:-1], training_frame = h2o_train_ind, validation_frame = h2o_valid_ind)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [26]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_binomial.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.5, lambda = 2.368E-5 )",20,20,3,py_1_sid_bda7


In [27]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_binomial.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.4989949,0.097696476,0.5488712,0.3687558,0.45590153,0.6287332,0.49271262
1,auc,0.5636197,0.006893701,0.56173456,0.55975866,0.5575829,0.5752457,0.56377655
2,aucpr,0.11421678,0.0021531337,0.11185258,0.11325577,0.11505778,0.11748842,0.113429345
3,err,0.5010051,0.097696476,0.45112875,0.6312442,0.54409844,0.3712668,0.5072874
4,err_count,8081.4,1603.9016,7214.0,10223.0,8754.0,5967.0,8249.0
5,f0point5,0.1330819,0.003528048,0.13250056,0.12905629,0.13078707,0.13797529,0.13509025
6,f1,0.18760657,0.0022391086,0.18541102,0.18729629,0.18658242,0.18738934,0.19135378
7,f2,0.319057,0.019092575,0.30866984,0.34133056,0.3254035,0.29194602,0.32793495
8,lift_top_group,1.4048964,0.2556131,1.5521312,1.090106,1.3495597,1.7537016,1.2789838
9,logloss,0.31119063,0.005265177,0.30661613,0.3148433,0.31579804,0.3044183,0.31427735


In [28]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_binomial._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-2.534238,-2.279643
1,LicAge,-0.000312,-0.049906
2,Gender,0.010311,0.004998
3,MariStat,-0.066143,-0.023827
4,DrivAge,-0.001085,-0.016265
5,HasKmLimit,-0.366059,-0.114316
6,BonusMalus,0.006341,0.097335
7,OutUseNb,0.061289,0.042639
8,RiskArea,0.009015,0.019975
9,VehUsage_Private,-0.146273,-0.069170


In [29]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_binomial.coef_norm()
for x in range(len(glm_binomial.cross_validation_models())):
    pmodels[x] = glm_binomial.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
Intercept,-2.27964,-2.27412,-2.28467,-2.28621,-2.27023,-2.28439
LicAge,-0.04991,-0.06056,-0.02989,-0.06488,-0.04887,-0.04591
Gender,0.00500,-0.00814,0.00592,0.00800,0.00641,0.01365
MariStat,-0.02383,-0.02894,-0.01516,-0.02738,-0.02649,-0.02192
DrivAge,-0.01626,-0.00039,-0.01926,0.00160,-0.04582,-0.03731
HasKmLimit,-0.11432,-0.11996,-0.10425,-0.11407,-0.10975,-0.12427
BonusMalus,0.09734,0.09806,0.10260,0.10616,0.08156,0.09762
OutUseNb,0.04264,0.04145,0.04063,0.04449,0.04107,0.04579
RiskArea,0.01997,0.02284,0.02115,0.02359,0.01745,0.01507
VehUsage_Private,-0.06917,-0.06312,-0.07678,-0.07634,-0.06413,-0.06078


In [30]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

ind_train_pred = glm_binomial.predict(h2o_train_ind).as_data_frame()
ind_valid_pred = glm_binomial.predict(h2o_valid_ind).as_data_frame()
ind_test_pred = glm_binomial.predict(h2o_test_ind).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [31]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [32]:
# Выведем импортированные выше метрики классификации для обучающей, валидационной и тестовой выборок

accuracy_score(y_train_ind, ind_train_pred['predict']), \
accuracy_score(y_valid_ind, ind_valid_pred['predict']), \
accuracy_score(y_test_ind, ind_test_pred['predict'])

(0.4550913060738388, 0.45608376339817386, 0.4540988487495038)

In [33]:
f1_score(y_train_ind, ind_train_pred['predict']), \
f1_score(y_valid_ind, ind_valid_pred['predict']), \
f1_score(y_test_ind, ind_test_pred['predict'])

(0.18614044839725774, 0.18915520047344284, 0.18311427934952104)

In [34]:
print(f"{confusion_matrix(y_train_ind, ind_train_pred['predict'])} \n"  \
f"{confusion_matrix(y_valid_ind, ind_valid_pred['predict'])} \n" \
f"{confusion_matrix(y_test_ind, ind_test_pred['predict'])}")

[[31661 41312]
 [ 2612  5023]] 
[[15825 20629]
 [ 1293  2557]] 
[[15836 20683]
 [ 1319  2466]]


Какие проблемы вы здесь видите? Как можно улучшить данный результат?


Модель дает плохие результаты.
Проблемы: не сбалансированная выборка от носительно целевой переменной, в Confusion matrix видно много ложных срабатываний.
Как улучшить: ввести параметр stratify,  использовать другое распределение, нелениейную модель, почистить данные.

In [35]:
new_model_path = h2o.save_model(glm_binomial)

In [36]:
new_model_path

'/Users/root1/Desktop/GU/ml-in-b/ml-in-b/GLM_model_python_1582117943665_1'